<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="400" alt="cognitiveclass.ai logo">
</center>

# **Investigation relationships between exchange rate BTC/BUSD and SME, MACD, Stochastic Oscillator indicators**

## Lab 6. Knowledge Check

Estimated time needed: **1** hour

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    
Для Марії
### The tasks:
*   

</div>

### Objectives

After completing this lab you will be able to:

*   Be confident about your data analysis skills

<h3>Table of Contents</h3>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ol>
    <li>Import and Load Data</li>
    <li>Questions</li>
</ol>

</div>

----


## Dataset Description

### Context
Dataset contains historical changes of the ***BTC/BUSD*** parameters for the period from *11/11/2022 to 11/24/2022* with an *1-minute* aggregation time.

### Columns

#### Input columns
* ***Time*** - the timestamp of the record
* ***Open*** -  the price of the asset at the beginning of the trading period
* ***High*** -  the highest price of the asset during the trading period
* ***Low*** - the lowest price of the asset during the trading period.
* ***Close*** - the price of the asset at the end of the trading period
* ***Volume*** - the total number of shares or contracts of a particular asset that are traded during a given period

#### Target columns
* ***Price*** - the average price at which a particular asset has been bought or sold during a given period


## 1. Import and Load Data 


### Import Libraries


Import the libraries necessary to use in this lab. We can add some aliases to make the libraries easier to use in our code and set a default figure size for further plots. Ignore the warnings.


In [ ]:
!pip install --upgrade scikit-learn
!pip install mplfinance

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import seaborn as sns
from typing import List
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as  mpatches
%matplotlib inline
import mplfinance as fplt
from sklearn import set_config
set_config(display='diagram')
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, accuracy_score, mean_absolute_percentage_error
import itertools
from itertools import combinations
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

### Load Data 


We will use the same DataSet like in previous labs. Therefore next some steps will be the same.


First, we assign the URL of the dataset to <code>"path"</code>. 


In [ ]:
path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX026LEN/BTCBUSD_trades_1m.csv"

Then use the Pandas method <code>read_csv()</code> to load the data from the web address and set dataframe index column type to <strong>datetime</strong> using <code>pd.to_datetime()</code> method for correct time series analysis. 


In [ ]:
df = pd.read_csv(path)
df.set_index('Time', inplace=True)
df.index = pd.to_datetime(df.index)

df.head()

## 2. Questions


<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<b style="font-size: 2em; font-weight: 600;">Question #1:</b>

Calculate financial technical SAME, MACD, Stochastic Oscillator indicators.
    
</div>


<h4>Simple Moving Average (SMA)</h4>


$Simple\ Moving\ Average\ (SMA)$ calculates the average of a selected range of prices, usually closing prices, by the number of periods in that range.

<em>A Simple Moving Average</em> is customizable because it can be calculated for different numbers of time periods. This is done by adding the closing price for a number of time periods and then dividing this total by the number of time periods, which gives the average price over the time period.

The $SMA$ is calculated as follows:

$$
SAME = \frac{A_1 + A_2 +\ ... \ + A_n}{n},
$$
<center>where $A_n$ — the price of an asset at period $n$, $n$ — the number of total periods<center>
    
Read more about this indicator <em><a href="https://www.investopedia.com/terms/s/sma.asp?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">here</a></em>.


<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<strong><em>Hint:</em></strong> to calculate SMA use <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01'><em>df['Close'].rolling(window = period).mean()</em></a>.
    
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
def sma(data, n):
    sma = data.rolling(window = n).mean()
    return sma

df['SMA'] = sma(df['Close'], 10)
df[['SMA']]

<h4>Moving Average Convergence/Divergence (MACD)</h4>


$Moving\ Average\ Convergence/Divergence\ (MACD)$ is a trend-following momentum indicator that shows the relationship between two exponential moving averages ($EMAs$) of a security’s price. 

The $MACD$ line is calculated by subtracting the <em>26-period EMA</em> from <em>the 12-period EMA</em>:

$$
MACD = Period_{12}\ EMA\ -\ Period_{26}\ EMA,
$$
<center><center>
    
Read more about this indicator <a href="https://www.investopedia.com/terms/m/macd.asp?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">here</a>.
    
<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<strong><em>Hint:</em></strong> We had calculated <strong><em>EMA</em></strong> before using <em><a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">ewm(span=period, min_periods=period, adjust=False).mean()</a></em> method. For more details follow <a href="https://www.investopedia.com/ask/answers/122314/what-exponential-moving-average-ema-formula-and-how-ema-calculated.asp?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01"><em>link</em></a>.
    
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
def ema(s: pd.Series, period: int) -> pd.Series:
    """Return the exponential moving average (EMA).
    """
    return s.ewm(span=period, min_periods=period, adjust=False).mean()

df['MACD'] = ema(df['Close'], 12) - ema(df['Close'], 26)
df[['MACD']]

<h4>Stochastic Oscillator</h4>


A $Stochastic\ Oscillator$ is a momentum indicator comparing a particular closing price to a range of its prices over a certain period of time. 

$$
K = \frac{Close\ -\ Low_{p}}{High_{p}\ -\ Low_{p}} \times 100,
$$
<center>
where $Close$ — the last closing price, $Low$ — the lowest low for the time period $p$,
</center>
<center>
$High$ — the highest high for the time period $p$.
</center>

Read more about this indicator <em><a href="https://www.investopedia.com/terms/s/stochasticoscillator.asp?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">here</a></em>.


<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<strong><em>Hint:</em></strong> use the following function to calculate:
*   $Low_{p}$ − <em><a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">df['Low'].rolling(p).min()</a></em>
*   $High_{p}$ − <em><a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">df['High'].rolling(p).max()</a></em>
    
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
def oscillator(pd: pd.DataFrame, period: int):
    """Return the Stochastic Oscillator.
    """
    # Adds a "n_high" column with max value of previous k periods
    high = df['High'].rolling(period).max()
    # Adds an "n_low" column with min value of previous k periods
    low = df['Low'].rolling(period).min()

    # Uses the min/max values to calculate the %k (as a percentage)
    k = (df['Close'] - low) * 100 / (high - low)
    return k

df['SO'] = oscillator(df, 10)
df[['SO']]

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<strong><em>Attention:</em></strong>
    All of obtained indicator values includes NaN due to specific period. We need to remove missing data.
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
# Remove NaN values in dataframe 
df.dropna(inplace=True)
df

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<b style="font-size: 2em; font-weight: 600;">Question #2:</b>

Perform resampling with a 1-day inteval and plot a corresponding candlestick.
</div>


<em><strong>Note:</strong></em> 
*  to keep the initial dataframe unchanged use <code><a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">df.copy()</a></code> method for dataframe copying
*  use <code><a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">df.resample()</a></code> method for resampling
*  to plot a candlestick use <code><a href="https://github.com/matplotlib/mplfinance/blob/master/examples/marketcolor_overrides.ipynb">mpf.plot(df, style='charles', type='candle')</a></code>.


In [ ]:
# Write your code below and press Shift+Enter to execute
# Resampling with 1-day interval
df_candlestick = df.copy()

df_candlestick = df_candlestick.loc[:, 'Price':'Low'].resample("1D").agg({
    'Open': 'first',
    'High': 'max',
    'Close': 'last',
    'Low': 'min',
    'Volume': 'sum',
    'Count': 'sum',
    'Price': 'mean'
})
df_candlestick.head()

In [ ]:
# Write your code below and press Shift+Enter to execute
# Plot a candlestick
df_candlestick.columns = ['open', 'high', 'low', 'close', 'volume', 'count', 'price']
fplt.plot(
            df_candlestick,
            type='candle',
            style='charles',
            title='BTCBUSD',
            ylabel='Price (BUSD)'
        )

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<b style="font-size: 2em; font-weight: 600;">Question #3:</b>

*    Create categorical variables for each of indicators and price.
*    Make separate columns in the dataframe for them.
*    Check the datatype of these columns and convert them to datatype <em>'category'</em>.
</div>


<em><strong>Note:</strong><em>
*  consider making three categories <strong><em>Low, Medium, High</em></strong>
*  to make bins of equal size use numpy's method <code><a href="https://numpy.org/doc/stable/reference/generated/numpy.linspace.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">numpy.linspace(start, stop, num)</a></code>
*  apply the function <code><a href="https://pandas.pydata.org/docs/reference/api/pandas.cut.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01&highlight=cut#pandas.cut">df.cut()</a></code> to determine what each binned value belongs to
*  for converting to needed datatype use <code><a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">df[[columns]].astype('category')</a></code> method


In [ ]:
# Write your code below and press Shift+Enter to execute
# List of categories
group_names = ["Low", "Medium", "High"]

# Write function responsible of creating categories
def to_categorical(column: pd.Series, categories: List[str]) -> pd.Series:
    """ Return categorical variables for a given dataframe column.
    """
    bins = np.linspace(min(column), max(column), len(categories) + 1)
    return pd.cut(column, bins, labels=categories, include_lowest=True)

# Create categories for indicators
indicators = ['SMA', 'MACD', 'SO', 'Price']
for indicator in indicators:
    df[f'{indicator}_binned'] = to_categorical(df[indicator], group_names)

# Filter display categorical columns
df.filter(regex='_binned').head()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<b style="font-size: 2em; font-weight: 600;">Question #4:</b>

Find a correlation between BTC price and obtained indicators. Build a heatmap.
</div>


<em><strong>Note:</strong></em>
*   use <code><a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">df[[columns]].corr()</a></code> to calculate a correlation for specific columns
*   use <code><a href="">seaborn.heatmap()</a></code> to visualize a correlation heatmap


In [ ]:
# Write your code below and press Shift+Enter to execute
# Find correlation
corr = df[['Price', 'SMA', 'MACD', 'SO']].corr()
corr

In [ ]:
# Write your code below and press Shift+Enter to execute
# Build a heatmap
sns.heatmap(corr, annot=True)

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<b style="font-size: 2em; font-weight: 600;">Question #5:</b>

Plot a scatterplot with the fitted regression line for each indicator and BTC price.
</div>


<em><strong>Note:</strong></em>
*    use <code><a href="https://seaborn.pydata.org/generated/seaborn.regplot.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">seaborn.regplot(x, y, data)</a></code> method to plot data and a linear regression model fit
*    consider using <code><a href="https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.xlabel.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">plt.xlabel()</a></code> and <code><a href="https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.ylabel.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">plt.ylabel()</a></code> to set label for axises
*    do <em>NOT</em> forget to use <code><a href="https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.show.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">plt.show()</a></code> method to display all figures


In [ ]:
# Write your code below and press Shift+Enter to execute
def regplot(pd: pd.DataFrame, x: str, y: str) -> None:
    """ Return data plot and a linear regression model fit.
    """
    sns.regplot(x=x, y=y, data=pd)
    plt.xlabel(f"{x}, BUSD")
    plt.ylabel(f"{y}, BUSD")
    plt.show()
    

indicators = ['SMA', 'MACD', 'SO']
for indicator in indicators:
    regplot(df, 'Price', indicator)

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<b style="font-size: 2em; font-weight: 600;">Question #6:</b>

For each pair of price and indicator:
*   calculate Pearson Correlation and P-value  
*   perform Durbin-Watson Test
*   perform Granger Causality Test
</div>


<em><strong>Note:</strong></em>
*   use <code><a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">scipy.stats.pearsonr(x, y)</a></code> method for <em>Pearson Correlation</em> and <em>P-value</em> calculation
*   you can also use wriiten in the previous modules functions to perform <em>Durbin-Watson</em> and <em>Granger Causality Test</em>


In [ ]:
# Write your code below and press Shift+Enter to execute
# Calculate Pearson Correlation and P-value for each of indicators and price 
df_stats = pd.DataFrame({"indicator":[], "Pearson": [], "P-value": []})
pd.options.display.float_format = '{:.3f}'.format

indicators = ['SMA', 'MACD', 'SO']
for indicator in indicators:
    pearson_coef, p_value = stats.pearsonr(df['Price'], df[indicator])
    df_stats.loc[len(df_stats.index)] = [indicator, pearson_coef, p_value]

df_stats.set_index('indicator', inplace=True)
df_stats

In [ ]:
# Write your code below and press Shift+Enter to execute
# Perform Durbin Watson Test
variables = ['Price', 'SMA', 'MACD', 'SO']

cols = [f"{variable}_dep" for variable in variables]
idxs = [f"{variable}_ind" for variable in variables]

dw_df = pd.DataFrame(columns=cols, index=idxs)

for (curr1, curr2) in itertools.permutations(variables, 2):
    
    X = df[f"{curr1}"]  # independent
    y = df[f"{curr2}"] # dependent
    # to get intercept
    X = sm.add_constant(X)
    # fit the regression model
    reg = sm.OLS(y, X).fit()
    dw = durbin_watson(resids=np.array(reg.resid))
    dw_df.loc[f"{curr1}_ind", f"{curr2}_dep"] = dw
    
np.fill_diagonal(dw_df.values, ' ')
dw_df

In [ ]:
# Write your code below and press Shift+Enter to execute
# Perform Granger-Causality Test
def grangers_causation_matrix(data, maxlag, variables, test='ssr_chi2test', verbose=False):
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df


cols = ['Price_y'] + [f"{indicator}_y" for indicator in indicators]
idxs = ['Price_x'] + [f"{indicator}_x" for indicator in indicators]

df_granger = pd.DataFrame(columns=cols, index=idxs)

for (curr1, curr2) in itertools.permutations(['Price'] + indicators, 2):
    df_test = df[[f"{curr1}", f"{curr2}"]]
    res_df = grangers_causation_matrix(df_test, 1, variables=df_test.columns)
    p1 = res_df[f"{curr1}_x"][f"{curr2}_y"]
    p2 = res_df[f"{curr2}_x"][f"{curr1}_y"]
    df_granger.loc[f"{curr1}_x", f"{curr2}_y"] = p1
    df_granger.loc[f"{curr2}_x", f"{curr1}_y"] = p2

# replace diagonal values with space char
np.fill_diagonal(df_granger.values, ' ')

df_granger

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<b style="font-size: 2em; font-weight: 600;">Question #7:</b>

Create a Multiple Linear Regression model with <em>indicators as input values</em> and <em>price as an output value.</em> 
    
Fit the model and make a prediction.

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
# Create Multiple Linear Regression
lm = LinearRegression()

# Fit the model
Z = df[['SMA', 'MACD', 'SO']]
lm.fit(Z, df['Price'])

# Make prediction
Y_hat = lm.predict(Z)

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<b style="font-size: 2em; font-weight: 600;">Question #8:</b>

Plot Multiple Linear Regression. 
</div>


In [ ]:
# Write your code below and press Shift+Enter to execute
# Plot Multiple Linear Regression
plt.figure(figsize=(8, 6))

ax1 = sns.distplot(df['Price'], hist=False, color="tab:orange", label="Actual Value")
sns.distplot(Y_hat, hist=False, color="tab:blue", label="Fitted Values" , ax=ax1)

handles = [mpatches.Patch(facecolor=plt.cm.Reds(100), label="Price"),
           mpatches.Patch(facecolor=plt.cm.Blues(100), label="Fitted values")]
plt.legend(handles=handles)

plt.title('Actual vs Fitted Values for Price')
plt.xlabel('Price')
plt.ylabel('Proportion of cryptocurrency')

plt.show()
plt.close()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<b style="font-size: 2em; font-weight: 600;">Question #9:</b>

For Multiple Linear Regression model calculate $R^2$, <em>MSE, MAPE</em> metrics.
</div>


<strong><em>Note:</em></strong> to calculate metrics above use the following functions:
*   $R^2$ − <code><a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">r2_score(y, y_pred)</a></code>
*   <em>MSE</em> − <code><a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">mean_squared_error(y, y_pred)</a></code>
*   <em>MAPE</em> − <code><a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_percentage_error.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">mean_absolute_percentage_error(y, y_pred)</a></code>


In [ ]:
# Write your code below and press Shift+Enter to execute
# Calculate metrics
def calculate_metrics(y_pred, y):
    r_squared = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    mape = mean_absolute_percentage_error(y, y_pred)
    
    print(f'The R-square value is: {r_squared:.4f}')
    print(f'The MSE value is: {mse:.4f}')
    print(f'The MAPE is: {mape:.4f}')

calculate_metrics(Y_hat, df['Price'])

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<b style="font-size: 2em; font-weight: 600;">Question #10:</b>

Encode categorical features <em>'SMA_binned', 'MACD_binned', 'SO_binned', 'Price_binned'</em> as an integer array in order to use them in classification.
</div>


<strong><em>Note:</em></strong> to do this task you can use:
*   <em>[LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX01KTEN2525-2023-01-01)</em>
*   <em>[OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX01KTEN2525-2023-01-01)</em>


In [ ]:
# Write your code below and press Shift+Enter to execute
# Encode categorical features
X = df[['SMA_binned', 'MACD_binned', 'SO_binned']]  #input columns
y = df['Price_binned']  #target column 

col_cat = ['SMA_binned', 'MACD_binned', 'SO_binned']
X = df[col_cat]
oe = OrdinalEncoder()
oe.fit(X[col_cat])
X_cat_enc = oe.transform(X[col_cat])

x_enc = pd.DataFrame(X_cat_enc)
x_enc.columns = col_cat

le = LabelEncoder()
le.fit(y)
y_enc = le.transform(y)
y_enc = pd.Series(y_enc)
y_enc.columns = y.name

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<b style="font-size: 2em; font-weight: 600;">Question #11:</b>

Split dataframe into train and test subsets.
</div>


<em><strong>Note:</strong></em> use <em><a href="">train_test_split(X, y, test_size)</a></em> to split the dataframe.


In [ ]:
# Write your code below and press Shift+Enter to execute
# Split into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(x_enc, y_enc, test_size=0.33, shuffle=False)

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<b style="font-size: 2em; font-weight: 600;">Question #12:</b>

Build a Decision Tree model and calculate its accuracy.
</div>


<em><strong>Note:</strong></em> use the following links to complete this question:
*   <a href="https://scikit-learn.org/stable/modules/tree.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01"><em>Skicit-leran Decision Tree</em></a>
*   <code><a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">accuracy_score()</a></code> to calculate accuracy


In [ ]:
# Write your code below and press Shift+Enter to execute
# Decision Tree
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
yhat = model.predict(X_test)

accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %.2f' % (accuracy*100))

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

<b style="font-size: 2em; font-weight: 600;">Question #13:</b>

Plot a Decision Tree.
</div>


<em><strong>Note:</strong></em> to plot a Decision Tree use one of the following methods:

*   <code><a href="https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX01KTEN2525-2023-01-01">plot_tree()</a></code>
*   <code><a href="https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">export_graphviz()</a></code>


In [ ]:
# Write your code below and press Shift+Enter to execute
# Plot Decision Tree
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(model,
               feature_names = x_enc.columns, 
               class_names = y.unique(),
               filled = True)

### Thank you for completing this lab!

## Authors
<a href="https://author.skills.network/instructors/yaryna_beida?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">Yaryna Beida</a>

<a href="https://author.skills.network/instructors/yaroslav_vyklyuk_2?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">Prof. Yaroslav Vyklyuk, DrSc, PhD</a>

<a href="https://author.skills.network/instructors/mariya_fleychuk?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX026LEN2588-2023-01-01">Prof. Mariya Fleychuk, DrSc, PhD</a>



## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
|2023-04-01|1.0| Yaryna Beida | Lab created |

<hr>

## <h3 align="center"> © IBM Corporation 2023. All rights reserved. <h3/>
